In [16]:
from matpower import start_instance
from oct2py import octave

import numpy as np
import pandas as pd


def read(file_path: str, sheet_name: str):
    df = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl')
    return df.to_numpy()

file_path = 'ac_case25.xlsx'
BUS = read(file_path, 'bus')
GEN = read(file_path, 'generator')
TR = read(file_path, 'transformer')
LINE = read(file_path, 'branch')

In [17]:
BUS

array([[1, 'PV', 108.0, 22.0, 1, 1, 0, 1.05, 0.95],
       [2, 'PV', 97.0, 20.0, 1, 1, 0, 1.05, 0.95],
       [3, 'PQ', 207.0, 42.55, 1, 1, 0, 1.05, 0.95],
       [4, 'PQ', 74.0, 15.0, 1, 1, 0, 1.05, 0.95],
       [5, 'PQ', 71.0, 14.0, 1, 1, 0, 1.05, 0.95],
       [6, 'PQ', 136.0, 28.0, 2, 1, 0, 1.05, 0.95],
       [7, 'PV', 125.0, 25.0, 2, 1, 0, 1.05, 0.95],
       [8, 'PQ', 136.8, 28.0, 2, 1, 0, 1.05, 0.95],
       [9, 'PQ', 175.0, 36.0, 1, 1, 0, 1.05, 0.95],
       [10, 'PQ', 156.0, 32.0, 2, 1, 0, 1.05, 0.95],
       [11, 'PQ', 0.0, 0.0, 3, 1, 0, 1.05, 0.95],
       [12, 'PQ', 0.0, 0.0, 3, 1, 0, 1.05, 0.95],
       [13, 'Swing', 265.0, 54.0, 3, 1, 0, 1.05, 0.95],
       [14, 'PV', 194.0, 39.0, 3, 1, 0, 1.05, 0.95],
       [15, 'PV', 253.6, 51.2, 4, 1, 0, 1.05, 0.95],
       [16, 'PV', 115.0, 23.0, 4, 1, 0, 1.05, 0.95],
       [17, 'PQ', 0.0, 0.0, 4, 1, 0, 1.05, 0.95],
       [18, 'PV', 333.0, 68.0, 4, 1, 0, 1.05, 0.95],
       [19, 'PQ', 208.15, 42.55, 3, 1, 0, 1.05, 0.95],
       [

In [18]:
columns = ['BUS', 'Type', 'Pload(MW)', 'Qload(MVAR)', 'area', 'Vm(pu)', 'Va(degree)', 'maxVm', 'minVm']
BUS = pd.DataFrame(BUS, columns=columns)

BUS = pd.DataFrame(BUS)

# 'Type' 열에서 PQ, PV, Swing을 1, 2, 3으로 매핑
type_mapping = {'PQ': 1, 'PV': 2, 'Swing': 3}
BUS['Type'] = BUS['Type'].map(type_mapping)

zero_array = np.zeros(len(BUS))
BUS.insert(5, "Gs", zero_array)
BUS.insert(6, "Bs", zero_array)


col_name = BUS.columns[4]      # 이동할 열 이름
BUS.insert(6,                  # 삽입할 위치 (7번째, 0-based index 6)
           col_name,          # 열 이름
           BUS.pop(col_name)  # pop으로 꺼낸 열 데이터
          )

one_array=np.ones(len(BUS))
BUS.insert(9, "baseKV", one_array)
BUS.insert(10, "zone", one_array)

BUS

,BUS,Type,Pload(MW),Qload(MVAR),Gs,Bs,area,Vm(pu),Va(degree),baseKV,zone,maxVm,minVm
0,1,2,108.0,22.0,0.0,0.0,1,1,0,1.0,1.0,1.05,0.95
1,2,2,97.0,20.0,0.0,0.0,1,1,0,1.0,1.0,1.05,0.95
2,3,1,207.0,42.55,0.0,0.0,1,1,0,1.0,1.0,1.05,0.95
3,4,1,74.0,15.0,0.0,0.0,1,1,0,1.0,1.0,1.05,0.95
4,5,1,71.0,14.0,0.0,0.0,1,1,0,1.0,1.0,1.05,0.95
5,6,1,136.0,28.0,0.0,0.0,2,1,0,1.0,1.0,1.05,0.95
6,7,2,125.0,25.0,0.0,0.0,2,1,0,1.0,1.0,1.05,0.95
7,8,1,136.8,28.0,0.0,0.0,2,1,0,1.0,1.0,1.05,0.95
8,9,1,175.0,36.0,0.0,0.0,1,1,0,1.0,1.0,1.05,0.95
9,10,1,156.0,32.0,0.0,0.0,2,1,0,1.0,1.0,1.05,0.95


In [19]:
m = start_instance()

mpc = {
    'version': '2',
    'baseMVA': 100,
    'bus': BUS.astype(float),     # (n_bus × 13)
    'gen': GEN.astype(float),     # (n_gen × 21)
    'branch': LINE.astype(float), 
}    

In [20]:
mpc['bus'] = BUS.to_numpy().astype(float)
mpc['bus']

array([[  1.  ,   2.  , 108.  ,  22.  ,   0.  ,   0.  ,   1.  ,   1.  ,
          0.  ,   1.  ,   1.  ,   1.05,   0.95],
       [  2.  ,   2.  ,  97.  ,  20.  ,   0.  ,   0.  ,   1.  ,   1.  ,
          0.  ,   1.  ,   1.  ,   1.05,   0.95],
       [  3.  ,   1.  , 207.  ,  42.55,   0.  ,   0.  ,   1.  ,   1.  ,
          0.  ,   1.  ,   1.  ,   1.05,   0.95],
       [  4.  ,   1.  ,  74.  ,  15.  ,   0.  ,   0.  ,   1.  ,   1.  ,
          0.  ,   1.  ,   1.  ,   1.05,   0.95],
       [  5.  ,   1.  ,  71.  ,  14.  ,   0.  ,   0.  ,   1.  ,   1.  ,
          0.  ,   1.  ,   1.  ,   1.05,   0.95],
       [  6.  ,   1.  , 136.  ,  28.  ,   0.  ,   0.  ,   2.  ,   1.  ,
          0.  ,   1.  ,   1.  ,   1.05,   0.95],
       [  7.  ,   2.  , 125.  ,  25.  ,   0.  ,   0.  ,   2.  ,   1.  ,
          0.  ,   1.  ,   1.  ,   1.05,   0.95],
       [  8.  ,   1.  , 136.8 ,  28.  ,   0.  ,   0.  ,   2.  ,   1.  ,
          0.  ,   1.  ,   1.  ,   1.05,   0.95],
       [  9.  ,   1.  , 175.  , 

In [21]:
GEN

array([[  1.   ,   1.   ,  10.   ,   0.   ,  10.   ,   0.   ,   1.035,
        100.   ,   1.   ],
       [  2.   ,   1.   ,  10.   ,   0.   ,  10.   ,   0.   ,   1.035,
        100.   ,   1.   ],
       [  3.   ,   1.   ,  76.   ,   0.   ,  30.   , -25.   ,   1.035,
        100.   ,   1.   ],
       [  4.   ,   1.   ,  76.   ,   0.   ,  30.   , -25.   ,   1.035,
        100.   ,   1.   ],
       [  5.   ,   2.   ,  10.   ,   0.   ,  10.   ,   0.   ,   1.035,
        100.   ,   1.   ],
       [  6.   ,   2.   ,  10.   ,   0.   ,  10.   ,   0.   ,   1.035,
        100.   ,   1.   ],
       [  7.   ,   2.   ,  76.   ,   0.   ,  30.   , -25.   ,   1.035,
        100.   ,   1.   ],
       [  8.   ,   2.   ,  76.   ,   0.   ,  30.   , -25.   ,   1.035,
        100.   ,   1.   ],
       [  9.   ,   7.   ,  80.   ,   0.   ,  60.   ,   0.   ,   1.025,
        100.   ,   1.   ],
       [ 10.   ,   7.   ,  80.   ,   0.   ,  60.   ,   0.   ,   1.025,
        100.   ,   1.   ],
       [ 11.   ,   7

In [22]:
columns = ['Id','Bus','PG(MW)','QG(MVAR)','QMAX(MVAR)','QMIN(MVAR)','Voltage setpoint(pu)','MBASE(MW)','STATUS']
GEN = pd.DataFrame(GEN, columns=columns)

GEN = pd.DataFrame(GEN)

GEN = GEN.drop(columns=['Id'])

zero_array = np.zeros(len(GEN))
GEN.insert(8, "Pmax", zero_array)
GEN.insert(9, "Pmin", zero_array)
GEN.insert(10, "Pc1", zero_array)
GEN.insert(11, "Pc2", zero_array)
GEN.insert(12, "Qc1min", zero_array)
GEN.insert(13, "Qc1max", zero_array)
GEN.insert(14, "Qc2min", zero_array)
GEN.insert(15, "Qc2max", zero_array)
GEN.insert(16, "ramp rate for load following/AGC (MW/min)", zero_array)
GEN.insert(17, "ramp rate for 10 minute reserves (MW)", zero_array)
GEN.insert(18, "ramp rate for 30 minute reserves (MW)", zero_array)
GEN.insert(19, "ramp rate for reactive power (2 sec timescale) (MVAr/min)", zero_array)
GEN.insert(20, "APF, area participation factor", zero_array)

GEN

,Bus,PG(MW),QG(MVAR),QMAX(MVAR),QMIN(MVAR),Voltage setpoint(pu),MBASE(MW),STATUS,Pmax,Pmin,...,Pc2,Qc1min,Qc1max,Qc2min,Qc2max,ramp rate for load following/AGC (MW/min),ramp rate for 10 minute reserves (MW),ramp rate for 30 minute reserves (MW),ramp rate for reactive power (2 sec timescale) (MVAr/min),"APF, area participation factor"
0,1.0,10.0,0.0,10.0,0.0,1.035,100.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,10.0,0.0,10.0,0.0,1.035,100.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,76.0,0.0,30.0,-25.0,1.035,100.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,76.0,0.0,30.0,-25.0,1.035,100.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.0,10.0,0.0,10.0,0.0,1.035,100.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2.0,10.0,0.0,10.0,0.0,1.035,100.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2.0,76.0,0.0,30.0,-25.0,1.035,100.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2.0,76.0,0.0,30.0,-25.0,1.035,100.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,7.0,80.0,0.0,60.0,0.0,1.025,100.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,7.0,80.0,0.0,60.0,0.0,1.025,100.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
mpc['gen'] = GEN.to_numpy().astype(float)
mpc['gen']

array([[  1.   ,  10.   ,   0.   ,  10.   ,   0.   ,   1.035, 100.   ,
          1.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
          0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ],
       [  1.   ,  10.   ,   0.   ,  10.   ,   0.   ,   1.035, 100.   ,
          1.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
          0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ],
       [  1.   ,  76.   ,   0.   ,  30.   , -25.   ,   1.035, 100.   ,
          1.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
          0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ],
       [  1.   ,  76.   ,   0.   ,  30.   , -25.   ,   1.035, 100.   ,
          1.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
          0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ],
       [  2.   ,  10.   ,   0.   ,  10.   ,   0.   ,   1.035, 100.   ,
          1.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
  

In [24]:
LINE

array([[1.0000e+00, 2.0000e+00, 2.6000e-03, 1.3900e-02, 4.6110e-01],
       [1.0000e+00, 3.0000e+00, 5.4600e-02, 2.1120e-01, 5.7200e-02],
       [1.0000e+00, 5.0000e+00, 2.1800e-02, 8.4500e-02, 2.2900e-02],
       [2.0000e+00, 4.0000e+00, 3.2800e-02, 1.2670e-01, 3.4300e-02],
       [2.0000e+00, 6.0000e+00, 4.9700e-02, 1.9200e-01, 5.2000e-02],
       [3.0000e+00, 9.0000e+00, 3.0800e-02, 1.1900e-01, 3.2200e-02],
       [3.0000e+00, 2.4000e+01, 2.3000e-03, 8.3900e-02, 0.0000e+00],
       [4.0000e+00, 9.0000e+00, 2.6800e-02, 1.0370e-01, 2.8100e-02],
       [5.0000e+00, 1.0000e+01, 2.2800e-02, 8.8300e-02, 2.3900e-02],
       [6.0000e+00, 1.0000e+01, 1.3900e-02, 6.0500e-02, 2.4590e+00],
       [7.0000e+00, 8.0000e+00, 1.5900e-02, 6.1400e-02, 1.6600e-02],
       [8.0000e+00, 9.0000e+00, 4.2700e-02, 1.6510e-01, 4.4700e-02],
       [8.0000e+00, 1.0000e+01, 4.2700e-02, 1.6510e-01, 4.4700e-02],
       [9.0000e+00, 1.1000e+01, 2.3000e-03, 8.3900e-02, 0.0000e+00],
       [9.0000e+00, 1.2000e+01, 2.

In [25]:
columns = ['f','t','tap']
TR = pd.DataFrame(TR, columns=columns)

TR = pd.DataFrame(TR)

TR['tap'] = 1 / TR['tap']

TR

,f,t,tap
0,3.0,24.0,0.970874
1,9.0,11.0,0.970874
2,9.0,12.0,0.970874
3,10.0,11.0,0.980392
4,10.0,12.0,0.980392


In [26]:
columns = ['f','t','r','x','b']
LINE = pd.DataFrame(LINE, columns=columns)

LINE = pd.DataFrame(LINE)

zero_array = np.zeros(len(LINE))
LINE.insert(5, "rateA", zero_array)
LINE.insert(6, "rateB", zero_array)
LINE.insert(7, "rateC", zero_array)
LINE.insert(8, "tap", zero_array)
LINE.insert(9, "shift", zero_array)
LINE.insert(10, "initial branch status", np.ones(len(LINE)))
LINE.insert(11, "minimum angle difference", zero_array)
LINE.insert(12, "maximum angle difference", zero_array)

LINE = LINE.merge(TR, on=['f', 't'], how='left', suffixes=('', '_new'))

# 2) `tap_new`가 존재하는 경우만 tap 값을 업데이트
LINE['tap'] = LINE['tap_new'].fillna(LINE['tap'])

# 3) 중간 컬럼 제거
LINE = LINE.drop(columns=['tap_new'])

LINE

,f,t,r,x,b,rateA,rateB,rateC,tap,shift,initial branch status,minimum angle difference,maximum angle difference
0,1.0,2.0,0.00260,0.013900,0.4611,0.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.0
1,1.0,3.0,0.05460,0.211200,0.0572,0.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.0
2,1.0,5.0,0.02180,0.084500,0.0229,0.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.0
3,2.0,4.0,0.03280,0.126700,0.0343,0.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.0
4,2.0,6.0,0.04970,0.192000,0.0520,0.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.0
5,3.0,9.0,0.03080,0.119000,0.0322,0.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.0
6,3.0,24.0,0.00230,0.083900,0.0000,0.0,0.0,0.0,0.970874,0.0,1.0,0.0,0.0
7,4.0,9.0,0.02680,0.103700,0.0281,0.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.0
8,5.0,10.0,0.02280,0.088300,0.0239,0.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.0
9,6.0,10.0,0.01390,0.060500,2.4590,0.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.0


In [27]:
mpc['branch'] = LINE.to_numpy().astype(float)
mpc['branch']

array([[1.00000000e+00, 2.00000000e+00, 2.60000000e-03, 1.39000000e-02,
        4.61100000e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [1.00000000e+00, 3.00000000e+00, 5.46000000e-02, 2.11200000e-01,
        5.72000000e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [1.00000000e+00, 5.00000000e+00, 2.18000000e-02, 8.45000000e-02,
        2.29000000e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [2.00000000e+00, 4.00000000e+00, 3.28000000e-02, 1.26700000e-01,
        3.43000000e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [2.00000000e+00, 6.00000000e+

In [28]:
ymat = m.makeYbus(mpc)

Y_mat_matpower = pd.DataFrame(ymat.todense())

Y_mat_matpower.index = range(1,ymat.shape[0]+1)
Y_mat_matpower.columns = range(1,ymat.shape[1]+1)

Y_mat_matpower


,1,2,3,4,5,6,7,8,9,10,...,16,17,18,19,20,21,22,23,24,25
1,17.011919-84.773857j,-13.001950+69.510427j,-1.147382+ 4.438224j,0.000000+ 0.000000j,-2.862587+11.095806j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,...,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j
2,-13.001950+69.510427j,16.180397-81.514918j,0.000000+ 0.000000j,-1.914911+ 7.396929j,0.000000+ 0.000000j,-1.263535+ 4.881262j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,...,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j
3,-1.147382+ 4.438224j,0.000000+ 0.000000j,6.280591-35.525214j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,-2.038434+ 7.875767j,0.000000+ 0.000000j,...,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,-0.336291+12.267301j,-2.748396+10.634653j
4,0.000000+ 0.000000j,-1.914911+ 7.396929j,0.000000+ 0.000000j,6.587185-25.430590j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,-2.336137+ 9.039455j,0.000000+ 0.000000j,...,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,-2.336137+ 9.039455j
5,-2.862587+11.095806j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,5.604049-21.689560j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,-2.741462+10.617154j,...,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j
6,0.000000+ 0.000000j,-1.263535+ 4.881262j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,4.870683-19.325938j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,-3.607148+15.700176j,...,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j
7,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,3.952500-15.254815j,-3.952500+15.263115j,0.000000+ 0.000000j,0.000000+ 0.000000j,...,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j
8,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,-3.952500+15.263115j,6.889095-26.564490j,-1.468297+ 5.677188j,-1.468297+ 5.677188j,...,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j
9,0.000000+ 0.000000j,0.000000+ 0.000000j,-2.038434+ 7.875767j,-2.336137+ 9.039455j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,-1.468297+ 5.677188j,6.535627-47.810549j,0.000000+ 0.000000j,...,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j
10,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,-2.741462+10.617154j,-3.607148+15.700176j,0.000000+ 0.000000j,-1.468297+ 5.677188j,0.000000+ 0.000000j,8.496280-55.513047j,...,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.

In [29]:
mpopt = m.mpoption('verbose', 2)

results = m.runpf(mpc, mpopt)


MATPOWER Version 8.0, 17-May-2024
Power Flow -- AC-polar-power formulation

 it    max residual        max ∆x
----  --------------  --------------
  0      5.866e+00           -
  1      4.713e-01       4.291e-01
  2      8.696e-03       2.581e-02
  3      7.973e-06       7.612e-04
  4      1.057e-11       8.191e-07
Newton's method converged in 4 iterations.
PF successful

Converged in 0.57 seconds
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses             25     Total Gen Capacity       0.0        -535.0 to 1776.0
Generators        33     On-line Capacity         0.0        -535.0 to 1776.0
Committed Gens    33     Generation (actual)   2865.9             464.3
Loads             18     Load                  2812.5             571.3
  Fixed           18       Fixed               2812.5        